In [43]:
# Import the necessary Packages for this software to run
import mediapipe as mp
import cv2
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

In [57]:
file_path = 'train_sortiert_new_format.csv'
df = pd.read_csv(file_path)

In [58]:
#split into target and data
target = df['label_id']
data = df.iloc[:,3:]
print("data: ",data.shape)
print("label:", target.shape)

#filter dataframe
filtered_df = df[df['label_id'] != 20]

#split into target and data
target = filtered_df['label_id']
data = filtered_df.iloc[:,3:]

#split into train and test data
X_train, X_test, y_train, y_test = train_test_split(data,target,random_state=1,test_size=0.2,shuffle=True)

data:  (16317, 2331)
label: (16317,)


In [59]:
#Create Model
clr_filtered = LogisticRegression(max_iter=2000, random_state=1)
clr_filtered.fit(X_train, y_train)

#predict training and new data
y_train_pred = clr_filtered.predict(X_train) 
y_test_pred = clr_filtered.predict(X_test)

#evaluation of the prediction
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Training Set Accuracy:", train_accuracy)
print("Test Set Accuracy:", test_accuracy)

Training Set Accuracy: 0.8083765112262522
Test Set Accuracy: 0.7638121546961326


In [60]:
#Create Table
df_columns = [] #'video_id', 'label_id', 'label']
for frame in range(1,38):
    for landmark in range (0,21):
        s1 = f'{"F"}{frame}{"_L"}{landmark}{"_X"}'
        s2 = f'{"F"}{frame}{"_L"}{landmark}{"_Y"}'
        s3 = f'{"F"}{frame}{"_L"}{landmark}{"_Z"}'
        df_columns.append(s1)
        df_columns.append(s2)
        df_columns.append(s3)


In [61]:
#Create Dataframe
df = pd.DataFrame(columns=df_columns)

In [62]:
#Show Dataframe
df.head()

,F1_L0_X,F1_L0_Y,F1_L0_Z,F1_L1_X,F1_L1_Y,F1_L1_Z,F1_L2_X,F1_L2_Y,F1_L2_Z,F1_L3_X,...,F37_L17_Z,F37_L18_X,F37_L18_Y,F37_L18_Z,F37_L19_X,F37_L19_Y,F37_L19_Z,F37_L20_X,F37_L20_Y,F37_L20_Z


In [51]:
# Use MediaPipe to draw the hand framework over the top of hands it identifies
drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands


def extract_data(image):
    # Use MediaPipe hand tracking with static_image_mode set to True
    with handsModule.Hands(static_image_mode=True, min_detection_confidence=0.7, max_num_hands=1) as hands:
        new_row=[]
        j = 0
        frame = cv2.resize(image, (640, 480))
        
        # Process the image and produce the hand framework overlay on top of the hand
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # In case the system sees multiple hands, this if statement deals with that and produces another hand overlay
        if results.multi_hand_landmarks:
            for handLandmarks in results.multi_hand_landmarks:
                drawingModule.draw_landmarks(frame, handLandmarks, handsModule.HAND_CONNECTIONS)
                i = 0
                last_idx = 0
                for idx, landmark in enumerate(handLandmarks.landmark):
                    while (i < idx):
                        new_row.append(0)
                        new_row.append(0)
                        new_row.append(0)
                        i = i+1

                    new_row.append(landmark.x)
                    new_row.append(landmark.y)
                    new_row.append(landmark.z)
                    i = idx+1
                    last_idx = idx
                if last_idx <= 20:
                    for i in range(last_idx+1, 21):
                        new_row.append(0)
                        new_row.append(0)
                        new_row.append(0)
        
        else:
            for i in range (0,63):
                new_row.append(0)
            j = j+1
        result = new_row.copy()
        new_row.clear()
        return result

In [67]:
images_dir = "C:\\Users\\wenni\\BV-Projekt-Echtzeit-Handerkennung\\14zip\\14"
images = []
for image in os.listdir(images_dir):
    image_path = os.path.join(images_dir, image)
    images.append(cv2.imread(image_path))


cap = cv2.VideoCapture(0)
row_arr = []
counter = 0
n = 37

for image in images:
    # ret, frame = cap.read()
    row_arr.extend(extract_data(image))
    counter += 1
    if counter > n:
        del row_arr[0:63]
    if counter == n:
        # model anwenden
        df.loc[0] = row_arr
        y_prediction = clr_filtered.predict(df)
        probabilities = clr_filtered.predict_proba(df)
        probabilities_in_percent = np.round(probabilities * 100, 2)
        print(y_prediction)
        print(probabilities_in_percent)

C:\Users\wenni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[5]
[[9.742e+01 5.000e-01 0.000e+00 4.000e-02 9.000e-02 3.000e-02 1.430e+00
  4.900e-01]]
